# Quesiton 1 - Data Loading and Cleaning

In [20]:
#get data to list
def turnToList(string):  
    f = open(string, 'r')
    x = f.readlines()
    #print(x)
    articleVocab = x[0]
    ls = articleVocab.split("', '")
    return ls

In [21]:
training_data = turnToList("Data/group1.train.txt")

In [22]:
import math

def makeBatches(data,batch_size):
    cuts = len(data)/batch_size
    cuts = math.ceil(cuts)
    final_list = [data[i:i+cuts] for i in range(0,len(data),cuts)]
    return final_list
    

In [23]:
bitches = makeBatches(training_data, 30)

In [24]:
print(len(bitches[28]))

156473


In [25]:
def sliding_windows(batch,n):
    window = [batch[i:i+n] for i in range(0,len(batch))]
    return window

In [26]:
batch1 = bitches[1]
w1 = sliding_windows(batch1,5)
batch2 = bitches[2]
w2 = sliding_windows(batch2,5)

In [27]:
print(w1[0:50])

[['the', 'rich', '.', '</s>', '<s>'], ['rich', '.', '</s>', '<s>', 'By'], ['.', '</s>', '<s>', 'By', '2018'], ['</s>', '<s>', 'By', '2018', ','], ['<s>', 'By', '2018', ',', 'the'], ['By', '2018', ',', 'the', 'richest'], ['2018', ',', 'the', 'richest', '400'], [',', 'the', 'richest', '400', 'Americans'], ['the', 'richest', '400', 'Americans', 'paid'], ['richest', '400', 'Americans', 'paid', 'lower'], ['400', 'Americans', 'paid', 'lower', 'taxes'], ['Americans', 'paid', 'lower', 'taxes', 'as'], ['paid', 'lower', 'taxes', 'as', 'a'], ['lower', 'taxes', 'as', 'a', 'share'], ['taxes', 'as', 'a', 'share', 'of'], ['as', 'a', 'share', 'of', 'income'], ['a', 'share', 'of', 'income', 'than'], ['share', 'of', 'income', 'than', 'any'], ['of', 'income', 'than', 'any', 'other'], ['income', 'than', 'any', 'other', 'income'], ['than', 'any', 'other', 'income', 'group'], ['any', 'other', 'income', 'group', 'including'], ['other', 'income', 'group', 'including', 'the'], ['income', 'group', 'including', 

In [28]:
#question 1 output
training_sequences = {}
training_sequences['Batch 1'] = [w1[i] for i in range(0,3)]
training_sequences['Batch 2'] = [w2[i] for i in range(0,3)]

In [29]:
print(training_sequences)

{'Batch 1': [['the', 'rich', '.', '</s>', '<s>'], ['rich', '.', '</s>', '<s>', 'By'], ['.', '</s>', '<s>', 'By', '2018']], 'Batch 2': [['to', 'strike', 'Soleimani', ',', 'who'], ['strike', 'Soleimani', ',', 'who', 'was'], ['Soleimani', ',', 'who', 'was', 'visiting']]}


## Question 2 - Bengio Model

In [42]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import random
import math
import time

Instructions for updating:
non-resource variables are not supported in the long term


In [43]:
#bring in pickle file for vocab dict
import pickle

file = open('Data/trainingDictPickle1_group1.pkl','rb')
vocab_dict = pickle.load(file)
file.close()

In [44]:
class Parameters():
    window_size = 5
    num_batches = 30

    vocab_size = len(vocab_dict)

    hidden_units = 50
    embeddings_dim = 60
    num_epochs = 20

    learning_rate = 0.5

    gpu_mem = 0.25

    tf_precision = tf.float32
    np_precision = np.float32

    init_scale = 0.5
    max_grad = 10.0

In [62]:
class Model(object):
    def __init__(self, params=Parameters()):
        #initialize input and output variables
        self._y = tf.placeholder(dtype=params.tf_precision,shape=(params.num_batches, params.vocab_size),name="Y")
        self._x = tf.placeholder(tf.int32,shape=(params.num_batches, params.window_size),name="X")
        #word features/embeddings
        self.C = tf.Variable(tf.truncated_normal(shape=(params.vocab_size, params.embeddings_dim),mean=-1,stddev=-1),dtype=params.tf_precision,name="C")
        #projection to output weight
        self.W = tf.Variable(tf.random_normal(shape=(params.vocab_size, (params.window_size) * params.embeddings_dim)),dtype=params.tf_precision,name="W")
        #hidden layer weight and bias
        self.H = tf.Variable(tf.random_normal(shape=(params.hidden_units, (params.window_size) * params.embeddings_dim)),dtype=params.tf_precision,name="H")
        self.d = tf.Variable(tf.random_normal(shape=(params.hidden_units,)),dtype=params.tf_precision,name="d")
        #hidden layer to output weight and bias
        self.U = tf.Variable(tf.random_normal((params.vocab_size, params.hidden_units)),name="U",dtype=params.tf_precision)
        self.b = tf.Variable(tf.random_normal(shape=(params.vocab_size, )),dtype=params.tf_precision,name="b")
        #do calculations
        # y = b + Wx + Utanh(d + Hx)
        # x = (C(w(t-1)), C(w(t-2), ..., C(w(t-n+1))), n == window_size
        with tf.name_scope('Projection_Layer'):
            x  = tf.nn.embedding_lookup(self.C, self._x) # (batch_size, window_size-1, emb_dim)
            x  = tf.reshape(x, shape=(batch_size, (params.window_size) * params.embeddings_dim))
        with tf.name_scope('Hidden_Layer'):
            Hx = tf.matmul(x, tf.transpose(self.H)) # (batch_size, hidden_size)
            o  = tf.add(self.d, Hx) # (batch_size, hidden_size)
            a  = tf.nn.tanh(o)  # (batch_size, hidden_size)
        with tf.name_scope('Output_Layer'):
            Ua = tf.matmul(a, tf.transpose(self.U)) # (batch_size, vocab_size)
            Wx = tf.matmul(x, tf.transpose(self.W)) # (batch_size, vocab_size)
            y_hat  = tf.nn.softmax(tf.clip_by_value(tf.add(self.b, tf.add(Wx, Ua)), 0.0, 10)) # (batch_size, vocab_size)
    #2 different ways to minimize loss/cost
        with tf.name_scope('Cost'):
            self.cost = tf.reduct_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self._y,logits=y_hat))
            self.perplexity = tf.exp(self.cost)
        self.optimizer = tf.train.GradientDescentOptimizer(params.learning_rate).minimize(self.cost)
    #loss function defined from github repo
    '''
        with tf.name_scope('Loss'):
            onehot_tgt = tf.one_hot(tf.squeeze(self._y), params.vocab_size)  # (batch_size, vocab_size)
            loss = -1 * tf.reduce_mean(tf.reduce_sum(tf.log(y) * onehot_tgt, 1)) # 乘 -1 -> maximize loss
            print(loss)
        
        optimizer = tf.train.AdagradOptimizer(params.learning_rate).minimize(loss)
        '''



In [47]:
def run_model(model,train_batch,validation_batch):
    #initialize and do GPU stuff
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=True)) as sess:
        initializer = tf.global_variables_initializer()
        initializer.run()
        step = 0
        avg_loss = 0
        for epoch in range(params.num_epochs):
            window = 0
            while window < params.vocab_size:
                
                for x_batch, y_batch in zip(train_batch,validation_batch):
                    if len(x_batch) != batch_size:
                        continue
                    feed_dict = {input_words: x_batch, output_word: y_batch}
                    fetches = [loss, optimizer]
                    Loss, _ = sess.run(fetches, feed_dict)
                    avg_loss += Loss
                    if step % 100 == 0:
                        print('Step {}, Loss: {}'.format(step, avg_loss / 100))
                        avg_loss = 0

                    step += 1

        print('Training Done.')
        word_embedding = C.eval()

SyntaxError: invalid syntax (<ipython-input-47-a776ab5578e3>, line 1)